In [1]:
!pip install -q --upgrade pip
!pip install -U jaxlib==0.4.4+cuda11.cudnn82 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q --upgrade jax[cuda11_cudnn805] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q git+https://www.github.com/google/neural-tangents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 72.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.8/112.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 82.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.


In [2]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
!pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cpu.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cpu
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 539.4/539.4 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.7/485.7 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 700.6/700.6 kB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 kB 39.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 kB 34.5 MB/s eta 0:00:00
  Created wheel for torch-geometric: filename=torch_geometric-2.2.0-py3-none-any.whl size=773

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK
from utils import *
from layers import two_wl_aggregation
import numpy
import neural_tangents as nt
from neural_tangents import stax
from jax import numpy as np
from jax import random
import jax
import shutil
import os
from jax.example_libraries import optimizers
from jax import jit, grad, vmap
from data_preperation import twl_sparse_pattern, twl_sparse_edge_features


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK


# Example to test if the pattern in the apply function behaves as expected (One graph)!

In [2]:
# define a toy 3x3 graph
# 0---2
# | /     
# 1 

nb_nodes = 3

edges = np.array([[0,0],
                   [1,1],
                   [2,2],
                   [0,1],
                   [0,2],
                   [1,2]])

node_features = np.array([[10],
                         [20],
                         [30],])

pattern, edge_list = twl_sparse_pattern(edges, nb_nodes)
edge_features = twl_sparse_edge_features(node_features, edge_list.shape[0], nb_nodes)

print(pattern.shape)
print(edge_features.shape)

edge_features

(18, 3)
(6, 2)


Array([[10.,  1.],
       [20.,  1.],
       [30.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]], dtype=float32)

In [3]:
# from layers import get_two_wl_aggregation_layer
# two_wl_aggregation_layer = get_two_wl_aggregation_layer("standard", 10)
# define new becaus need to not use relu to check results

L_branche = stax.serial(
    stax.Dense(1, parameterization="standard"),
    )

Gamma_branche = stax.serial(
    stax.Dense(1, parameterization="standard"),
    two_wl_aggregation(),
)

two_wl_aggregation_layer = stax.serial(
    stax.FanOut(2), 
    stax.parallel(L_branche, Gamma_branche),
    stax.FanInSum(),
    # stax.Relu(),
    )

init_fn, apply_fn, kernel_fn = stax.serial(
    two_wl_aggregation_layer,
)

key = random.PRNGKey(0)
key, subkey = jax.random.split(key)
_, params = init_fn(subkey, edge_features.shape)
params

[[(), ([(Array([[-0.77373135],
            [ 0.4843592 ]], dtype=float32), None)], [(Array([[-0.12870112],
            [ 0.0934424 ]], dtype=float32), None), ()]), ()]]

In [4]:
# set all weights to 0/1 for the dense
params[0][1][0][0]  = (np.zeros(params[0][1][0][0][0].shape), None) 
params[0][1][1][0]  = (np.ones(params[0][1][1][0][0].shape), None)

params

[[(), ([(Array([[0.],
            [0.]], dtype=float32), None)], [(Array([[1.],
            [1.]], dtype=float32), None), ()]), ()]]

In [5]:
# the convolution weights are all 1 => the output of the convlution is the sum of the output of the agg layer
# after the convolution we thus have np.sum(np.squeeze(edge_features), 1) = [11. 21. 31.  1.  1.  1.]
out = apply_fn(params, edge_features, pattern=pattern)

print(np.sum(np.squeeze(edge_features), 1))
print(np.squeeze(out))

# 0,0 -> 11
# 1,1 -> 21
# 2,2 -> 31
# 0,1 -> 1
# 0,2 -> 1
# 1,2 -> 1

# i,j i,a a,j 
# 0,0 0,1 1,0  1  1
# 0,0 0,0 0,0 11 11
# 0,0 0,2 2,0  1  1
#              23 23 -> 26

# 2,2 2,2 2,2 31 31
# 2,2 2,0 0,2  1  1
# 2,2 2,1 1,2  1  1
#              33 33 -> 66

[11. 21. 31.  1.  1.  1.]
[26. 46. 66. 36. 46. 56.]


In [6]:
# only checking the shape
out_kernel = kernel_fn(edge_features, edge_features, pattern=(pattern, pattern), nb_edges=(6,6))
print(out_kernel.ntk)

/usr/local/lib/python3.9/dist-packages/neural_tangents/_src/stax/branching.py:399: UserWarning: `FanIn` layers assume independent inputs which is not verified in the code. Please make sure to have at least one `Dense` / `Conv` / `GlobalSelfAttention` etc. layer in each branch.
  warnings.warn('`FanIn` layers assume independent inputs which is not verified'


[[ 537. 1037. 1537.  637.  837. 1037.]
 [1037. 2037. 3037. 1237. 1637. 2037.]
 [1537. 3037. 4537. 1837. 2437. 3037.]
 [ 637. 1237. 1837.  937. 1237. 1537.]
 [ 837. 1637. 2437. 1237. 1637. 2037.]
 [1037. 2037. 3037. 1537. 2037. 2537.]]


# Example (two graphs) to test if the full network behaves aas expected

In [7]:
# define a toy 3x3 graph
# 0---2
# | /     
# 1 

# define a toy 1x1 graph
# 3---4

nb_nodes = 5
edges = np.array([[0,0],
                   [1,1],
                   [2,2],
                   [0,1],
                   [0,2],
                   [1,2],
                   [3,3],
                   [3,4],
                  [4,4]])

graph_indx = np.array([[0],
                       [0],
                       [0],
                       [0],
                       [0],
                       [0],
                       [1],
                       [1],
                       [1]])

node_features = np.array([[10],
                         [20],
                         [30],
                         [40],
                        [50],])

pattern, edge_list = twl_sparse_pattern(edges, nb_nodes)
edge_features = twl_sparse_edge_features(node_features, edge_list.shape[0], nb_nodes)

graph_indx = np.squeeze(graph_indx)
print(pattern.shape)
print(edge_features.shape)
print(graph_indx.shape)
edge_features

(24, 3)
(9, 2)
(9,)


Array([[10.,  1.],
       [20.,  1.],
       [30.,  1.],
       [40.,  1.],
       [50.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]], dtype=float32)

In [8]:
from layers import index_aggregation
# two_wl_aggregation_layer = get_two_wl_aggregation_layer("standard", 10)
# define new becaus need to not use relu to check results

L_branche = stax.serial(
    stax.Dense(1, parameterization="standard"),
    )

Gamma_branche = stax.serial(
    stax.Dense(1, parameterization="standard"),
    two_wl_aggregation(),
)

two_wl_aggregation_layer = stax.serial(
    stax.FanOut(2), 
    stax.parallel(L_branche, Gamma_branche),
    stax.FanInSum(),
    # stax.Relu(),
    )

init_fn, apply_fn, kernel_fn = stax.serial(
    two_wl_aggregation_layer,
    index_aggregation(),
)

key = random.PRNGKey(0)
key, subkey = jax.random.split(key)
_, params = init_fn(subkey, edge_features.shape)
params

[[(), ([(Array([[-0.77373135],
            [ 0.4843592 ]], dtype=float32), None)], [(Array([[-0.12870112],
            [ 0.0934424 ]], dtype=float32), None), ()]), ()], ()]

In [9]:
# set all weights to 1 for the convolution
params[0][1][0][0]  = (np.zeros(params[0][1][0][0][0].shape), None) 
params[0][1][1][0]  = (np.ones(params[0][1][1][0][0].shape), None)

params

[[(), ([(Array([[0.],
            [0.]], dtype=float32), None)], [(Array([[1.],
            [1.]], dtype=float32), None), ()]), ()], ()]

In [10]:
# the dense weights are all 1 => the output of the convlution is the sum of the output of the agg layer
# after the twl layer we have [ 26.  46.  66.  84. 104.  36.  46.  56.  94.]
out = apply_fn(params, edge_features, pattern=pattern, graph_indx=graph_indx, nb_graphs=2)

print(np.sum(np.squeeze(edge_features), 1))
print(np.squeeze(out))

a = np.array([26., 46., 66., 84., 104., 36., 46., 56., 94.])
print(np.sum(a[:-3]))
print(np.sum(a[-3:]))

[11. 21. 31. 41. 51.  1.  1.  1.  1.]
[362. 196.   0.   0.   0.   0.   0.   0.   0.]
362.0
196.0


In [11]:
# only checking the shape
out_kernel = kernel_fn(edge_features, edge_features, pattern=(pattern, pattern), graph_indx=(graph_indx, graph_indx), nb_edges=(9,9), nb_graphs=(2,2))
print(out_kernel.ntk)

/usr/local/lib/python3.9/dist-packages/neural_tangents/_src/stax/branching.py:399: UserWarning: `FanIn` layers assume independent inputs which is not verified in the code. Please make sure to have at least one `Dense` / `Conv` / `GlobalSelfAttention` etc. layer in each branch.
  warnings.warn('`FanIn` layers assume independent inputs which is not verified'


[[[[132460.  59930.]
   [ 59930.  32665.]]]]


# Test full network

In [12]:
# define a toy 3x3 graph
# 0---2
# | /     
# 1 

# define a toy 1x1 graph
# 3---4

nb_nodes = 5
edges = np.array([[0,0],
                   [1,1],
                   [2,2],
                   [0,1],
                   [0,2],
                   [1,2],
                   [3,3],
                   [3,4],
                  [4,4]])

graph_indx = np.array([[0],
                       [0],
                       [0],
                       [0],
                       [0],
                       [0],
                       [1],
                       [1],
                       [1]])

node_features = np.array([[10],
                         [20],
                         [30],
                         [40],
                        [50],])

pattern, edge_list = twl_sparse_pattern(edges, nb_nodes)
edge_features = twl_sparse_edge_features(node_features, edge_list.shape[0], nb_nodes)

graph_indx = np.squeeze(graph_indx)
print(pattern.shape)
print(edge_features.shape)
print(graph_indx.shape)
edge_features

(24, 3)
(9, 2)
(9,)


Array([[10.,  1.],
       [20.,  1.],
       [30.,  1.],
       [40.,  1.],
       [50.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]], dtype=float32)

In [13]:
from layers import index_aggregation, get_two_wl_aggregation_layer
two_wl_aggregation_layer = get_two_wl_aggregation_layer("standard", 10)

init_fn, apply_fn, kernel_fn = stax.serial(
    two_wl_aggregation_layer,
    two_wl_aggregation_layer,
    two_wl_aggregation_layer,
    index_aggregation(),
    stax.Dense(1),
)

key = random.PRNGKey(0)
key, subkey = jax.random.split(key)
_, params = init_fn(subkey, edge_features.shape)

out_apply = apply_fn(params, edge_features, pattern=pattern, graph_indx=graph_indx, nb_graphs=2)
out_kernel = kernel_fn(edge_features, edge_features, pattern=(pattern, pattern), graph_indx=(graph_indx, graph_indx), nb_edges=(9,9), nb_graphs=(2,2))

print(np.squeeze(out_apply))
print(out_kernel.ntk)

/usr/local/lib/python3.9/dist-packages/neural_tangents/_src/stax/branching.py:399: UserWarning: `FanIn` layers assume independent inputs which is not verified in the code. Please make sure to have at least one `Dense` / `Conv` / `GlobalSelfAttention` etc. layer in each branch.
  warnings.warn('`FanIn` layers assume independent inputs which is not verified'


[395.15192 210.22028   0.        0.        0.        0.        0.
   0.        0.     ]
[[[[1.1888026e+08 6.1424736e+07]
   [6.1424736e+07 3.1748660e+07]]]]


In [14]:
pattern.shape

(24, 3)

# Example to test if the pattern in the kernel function behaves as expected!

In [15]:
from typing import Callable, Iterable, Optional, Sequence, Tuple, Union
from neural_tangents import Kernel
from neural_tangents._src.stax.requirements import (
    Bool,
    Diagonal,
    get_diagonal_outer_prods,
    layer,
    mean_and_var,
    requires,
    supports_masking,
)
from utils import row_wise_karthesian_prod


def kernel_fn(k, *,pattern, nb_edges: Tuple[Optional[int], Optional[int]] = (None, None),**kwargs):

        num_segments = int(np.prod(np.array(k.ntk.shape)))

        patterns = row_wise_karthesian_prod(pattern[0], pattern[1])

        nb_edges_graph_a = nb_edges[0]
        nb_edges_graph_b = nb_edges[1]

        e_i_j_ib_jb = np.ravel_multi_index(
            [patterns[:, 0], patterns[:, 3]], (nb_edges_graph_a, nb_edges_graph_b)
        )
        e_i_a_ib_ab = np.ravel_multi_index(
            [patterns[:, 1], patterns[:, 4]], (nb_edges_graph_a, nb_edges_graph_b)
        )
        e_i_a_ab_jb = np.ravel_multi_index(
            [patterns[:, 1], patterns[:, 5]], (nb_edges_graph_a, nb_edges_graph_b)
        )
        e_a_j_ib_ab = np.ravel_multi_index(
            [patterns[:, 2], patterns[:, 4]], (nb_edges_graph_a, nb_edges_graph_b)
        )
        e_a_j_ab_jb = np.ravel_multi_index(
            [patterns[:, 2], patterns[:, 5]], (nb_edges_graph_a, nb_edges_graph_b)
        )

        def agg(x):
          theta_i_a_ib_ab = jax.ops.segment_sum(
              np.take(x, e_i_a_ib_ab), e_i_j_ib_jb, num_segments
          )
          theta_i_a_ab_jb = jax.ops.segment_sum(
              np.take(x, e_i_a_ab_jb), e_i_j_ib_jb, num_segments
          )
          theta_a_j_ib_ab = jax.ops.segment_sum(
              np.take(x, e_a_j_ib_ab), e_i_j_ib_jb, num_segments
          )
          theta_a_j_ab_jb = jax.ops.segment_sum(
              np.take(x, e_a_j_ab_jb), e_i_j_ib_jb, num_segments
          )

          thetas_linear = np.array(
            [theta_i_a_ib_ab, theta_i_a_ab_jb, theta_a_j_ib_ab, theta_a_j_ab_jb]
          )
          theta_linear = np.sum(thetas_linear, 0)
          theta = np.reshape(theta_linear, x.shape)
          return theta

        ntk = agg(k.ntk)
        nngp = agg(k.nngp)
        
        return k.replace(ntk=ntk, nngp=nngp, is_gaussian=True, is_input=False)

In [16]:
# define a toy 2x graph
# 0---2 

toy_kernel_2_new = np.array(range(3*3))
toy_kernel_2_new = np.reshape(toy_kernel_2_new, (3,3))

toy_kernel_2_new = np.expand_dims(toy_kernel_2_new, 0)
toy_kernel_2_new = np.expand_dims(toy_kernel_2_new, 0)
toy_kernel_2_new = np.expand_dims(toy_kernel_2_new, 0)
toy_kernel_2_new = np.expand_dims(toy_kernel_2_new, 0)

toy_kernel_2_new

edge_list_2 = np.array([[0,0],
                      [1,1],
                      [0,1],
                      [1,0]])

pattern, edge_list = twl_sparse_pattern(edge_list_2, 2)

pattern

Array([[0, 0, 0],
       [1, 1, 1],
       [2, 2, 1],
       [2, 0, 2],
       [1, 2, 2],
       [0, 2, 2]], dtype=int32)

The output of the kernel function for the toy kernel ($\Theta_{(i,j,i',j')}^{(l}$) must be:
$$
    \Theta_{(i,j,i',j')}^{(l)}
    =
    \sum_{v_a \in \Gamma_G^r(v_i) \cap \Gamma_G^r(v_j)}
    \sum_{v_a' \in \Gamma_G^r(v_i') \cap \Gamma_G^r(v_j')}
    \Theta_{(i,a,i',a')}^{(l-1)}
    +
    \Theta_{(a,j,a',j')}^{(l-1)}
    +
    \Theta_{(i,a,a',j')}^{(l-1)}
    +
    \Theta_{(a,j,i',a')}^{(l-1)}
$$
We can calculate this manually with the following table (see excel file)

In [17]:
k = nt.Kernel(nngp=toy_kernel_2_new, 
          ntk=toy_kernel_2_new,
          cov1=None,
          cov2=None, 
          x1_is_x2=None, 
          is_gaussian=None, 
          is_reversed=None, 
          is_input=None, 
          diagonal_batch=None, 
          diagonal_spatial=None, 
          shape1=None, 
          shape2=None, 
          batch_axis=None, 
          channel_axis=None, 
          mask1=None, 
          mask2=None)

kernel_matrix = kernel_fn(k=k, pattern=(pattern, pattern), nb_edges=(3,3), nb_graphs=(1,1))

kernel_matrix.ntk

Array([[[[[[64, 72, 68],
           [88, 96, 92],
           [76, 84, 80]]]]]], dtype=int32)

# check if autograd works

In [18]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK
from utils import *
from layers import two_wl_aggregation
import numpy
import neural_tangents as nt
from neural_tangents import stax
from jax import numpy as np
from jax import random
import jax
import shutil
import os
from jax.example_libraries import optimizers
from jax import jit, grad, vmap
from data_preperation import twl_sparse_pattern, twl_sparse_edge_features

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK


In [19]:
# define a toy 3x3 graph
# 0---2
# | /     
# 1 

# define a toy 1x1 graph
# 3---4

nb_nodes = 5
edges = np.array([[0,0],
                   [1,1],
                   [2,2],
                   [0,1],
                   [0,2],
                   [1,2],
                   [3,3],
                   [3,4],
                  [4,4]])

graph_indx = np.array([[0],
                       [0],
                       [0],
                       [0],
                       [0],
                       [0],
                       [1],
                       [1],
                       [1]])

node_features = np.array([[10],
                         [20],
                         [30],
                         [40],
                        [50],])

pattern, edge_list = twl_sparse_pattern(edges, nb_nodes)
edge_features = twl_sparse_edge_features(node_features, edge_list.shape[0], nb_nodes)

Y = np.array([0,1])

# edge_features = np.transpose(np.array([edge_features[:,1], edge_features[:,1]]))


graph_indx = np.squeeze(graph_indx)
print(pattern.shape)
print(edge_features.shape)
print(graph_indx.shape)
pattern

(24, 3)
(9, 2)
(9,)


Array([[0, 0, 0],
       [1, 1, 1],
       [5, 5, 1],
       [2, 2, 2],
       [6, 6, 2],
       [7, 7, 2],
       [3, 3, 3],
       [4, 4, 4],
       [8, 8, 4],
       [5, 0, 5],
       [1, 5, 5],
       [6, 0, 6],
       [7, 5, 6],
       [2, 6, 6],
       [7, 1, 7],
       [6, 5, 7],
       [2, 7, 7],
       [8, 3, 8],
       [4, 8, 8],
       [0, 5, 5],
       [0, 6, 6],
       [5, 6, 7],
       [1, 7, 7],
       [3, 8, 8]], dtype=int32)

In [ ]:
from typing import Callable, Iterable, Optional, Sequence, Tuple, Union
from neural_tangents import Kernel
from neural_tangents._src.stax.requirements import (
    Bool,
    Diagonal,
    get_diagonal_outer_prods,
    layer,
    mean_and_var,
    requires,
    supports_masking,
)
from jax import numpy as np
import jax
from utils import row_wise_karthesian_prod
from neural_tangents import stax

@layer
@supports_masking(remask_kernel=False)
def index_aggregation():
    init_fn = lambda rng, input_shape: (input_shape, ())

    def apply_fn(
        params,
        inputs: np.ndarray,
        *,
        graph_indx: Optional[np.ndarray] = None,
        nb_graphs: Optional[int] = None,
        **kwargs
    ):

        # Can not use nb_graphs here as this gives an error when
        # jiting the apply function for gradient descent
        # thus need to use the input shape.
        # This implies, that the output shape is of the same 
        # size as the input. 
        # Only the first nb_graphs entries are non 0 thoug.
        # The hack to make this work is to only consider the
        # first nb_graphs entires for the loss calculation
        num_segments = inputs.shape[0]
        out = np.apply_along_axis(
            lambda x: jax.ops.segment_sum(x, graph_indx, num_segments),
            0,
            np.squeeze(inputs),
        )
        return out

    def kernel_fn(
        k: Kernel,
        *,
        graph_indx: Tuple[Optional[np.ndarray], Optional[np.ndarray]] = (None, None),
        nb_graphs: Tuple[Optional[int], Optional[int]] = (None, None),
        **kwargs
    ):
        num_segments = nb_graphs[0] * nb_graphs[1]

        def agg(x, kernel_graph_indx):
            agg_x = jax.ops.segment_sum(
                np.reshape(x, (-1)),
                kernel_graph_indx,
                num_segments,
            )
            agg_x = np.reshape(agg_x, nb_graphs)
            agg_x = np.expand_dims(agg_x, 0)
            agg_x = np.expand_dims(agg_x, 0)
            return agg_x

        k_prod_graph_indx = row_wise_karthesian_prod(
            np.expand_dims(graph_indx[0], 1), np.expand_dims(graph_indx[1], 1)
        )
        kernel_graph_indx = np.ravel_multi_index(
            [k_prod_graph_indx[:, 0], k_prod_graph_indx[:, 1]], nb_graphs
        )

        agg_ntk = agg(k.ntk, kernel_graph_indx)
        agg_nngp = agg(k.nngp, kernel_graph_indx)

        return k.replace(
            ntk=agg_ntk, nngp=agg_nngp, is_gaussian=True, is_input=False, channel_axis=1
        )

    return init_fn, apply_fn, kernel_fn

In [ ]:
from layers import get_two_wl_aggregation_layer
# two_wl_aggregation_layer = get_two_wl_aggregation_layer("standard", 3)

L_branche = stax.serial(
    stax.Dense(3, parameterization="standard"),
)

Gamma_branche = stax.serial(
    stax.Dense(3, parameterization="standard"),
    two_wl_aggregation(),
)

two_wl_aggregation_layer = stax.serial(
    stax.FanOut(2),
    stax.parallel(L_branche, Gamma_branche),
    stax.FanInSum(),
    stax.Relu(),
)

init_fn, apply_fn, kernel_fn = stax.serial(
    two_wl_aggregation_layer,
    # two_wl_aggregation_layer,
    # two_wl_aggregation_layer,
    # two_wl_aggregation_layer,
    # two_wl_aggregation_layer,
    # two_wl_aggregation_layer,
    index_aggregation(),
    stax.Dense(1, parameterization="standard"),
)

key = random.PRNGKey(1701)
key, subkey = jax.random.split(key)
_, params = init_fn(subkey, edge_features.shape)

In [ ]:
from train import cross_entropy
from functools import partial


learning_rate = 0.001

loss = jit(cross_entropy)
grad_loss = jit(grad(lambda params, x, y, graph_indx=graph_indx, **kwargs: loss(y, (apply_fn(params, x,  graph_indx=graph_indx, **kwargs))[:y.shape[0],:])))


@partial(jit, static_argnames=['nb_graphs'])
def update(state, params, x, y, nb_graphs, graph_indx, **kwargs):
  graph_indx = np.array(graph_indx)
  grads = grad_loss(params, x, y, nb_graphs=nb_graphs, graph_indx=graph_indx,**kwargs)
  return opt_apply(0, grads, state)

opt_init, opt_apply, get_params = optimizers.adam(learning_rate)
opt_apply = jit(opt_apply)
state = opt_init(params)

params = get_params(state)

state = opt_apply(
    0, grad_loss(params, edge_features, Y, pattern=pattern, nb_graphs=2, graph_indx=graph_indx), state
)

# state = update(state=state, params=params, x=edge_features, y=Y, pattern=pattern, nb_graphs=2, graph_indx=graph_indx)

In [ ]:
apply_fn(params, edge_features, pattern=pattern, nb_graphs=2, graph_indx=graph_indx)[:Y.shape[0],:]

# Minimal example

In [ ]:
init_fn, apply_fn, kernel_fn = stax.serial(
    index_aggregation(),
)

key = random.PRNGKey(1701)
key, subkey = jax.random.split(key)
_, params = init_fn(subkey, edge_features.shape)